In [1]:
import asyncio
import json
import os
from datetime import datetime

import aiohttp
import nest_asyncio
import numpy as np
import requests
import requests.auth
from dotenv import load_dotenv

In [2]:
load_dotenv()
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")

In [3]:
# async def fetch(url):
#     async with aiohttp.ClientSession() as session:
#         try:
#             async with session.get(url) as response:
#                 return await response.text()
#         except aiohttp.ClientError as e:
#             print(f"An error occurred: {e}")
#             return None


# async def main():
#     urls = [
#         "https://www.reddit.com/r/meme/top.json?t=week",
#     ]
#     tasks = [fetch(url) for url in urls]
#     responses = await asyncio.gather(*tasks)
#     for response in responses:
#         if response:
#             print(response)

# nest_asyncio.apply()

# loop = asyncio.get_event_loop()
# res = loop.run_until_complete(main())

In [4]:
client_auth = requests.auth.HTTPBasicAuth(REDDIT_CLIENT_ID, REDDIT_CLIENT_SECRET)
post_data = {"grant_type": "password", "username": "reddit_bot", "password": "snoo"}
headers = {"User-Agent": "ChangeMeClient/0.1 by YourUsername"}
response = requests.post(
    "https://www.reddit.com/api/v1/access_token",
    auth=client_auth,
    data=post_data,
    headers=headers,
)

In [5]:
response.json()

{'error': 'invalid_grant'}

In [5]:
params = {"t": "year", "limit": 100}
urls = []
for _ in range(5):
    response = requests.get("https://www.reddit.com/r/meme/top.json", params=params)
    data = response.json()
    if "error" in data:
        raise Exception()
    params["after"] = data["data"]["after"]
    children = data["data"]["children"]
    for i in range(len(children)):
        urls.append(data[i]["data"]["url"])

KeyError: 'data'

In [ ]:
with open("response.txt", "w") as f:
    json.dump(response.json(), f)

In [ ]:
unique_exts = [url.split(".")[-1] for url in urls]
np.unique(unique_exts)

In [ ]:
filter_urls = [url for url in urls if url.split(".")[-1] not in {"jpeg", "jpg", "png"}]
len(filter_urls), filter_urls

In [10]:
for i, url in enumerate(filter_urls):
    image = requests.get(url)
    file_name = url.split("/")[-1]
    with open(f"images/{file_name}", "wb") as f:
        f.write(image.content)

KeyboardInterrupt: 